In [5]:
!pip install pandas
!pip install sklearn
!pip install tensorflow
!pip install dlomix
!pip install numpy
!pip install matplotlib

In [6]:
import pandas as pd
# Importing the data
data = pd.read_csv('https://raw.githubusercontent.com/ProteomicsML/RetentionTime/main/datasets/ProteomeTools_RT/Small.csv')
# Normalize the data
data['Retention time'] = data['Retention time'] / data['Retention time'].max()

from sklearn.model_selection import train_test_split
# Split the data into training and test sets
train, test = train_test_split(data, test_size=0.2)
print('Training data point:', len(train), '\nTesting data point:', len(test))

Training data point: 80000 
Testing data point: 20000


In [8]:
import numpy as np
from dlomix.data import RetentionTimeDataset
# Setup parameters
sequence_length = 30
batch_size = 256
train_array = tuple([np.array(train['Sequence']), np.array(train['Retention time'])])
test_array = tuple([np.array(test['Sequence']), np.array(test['Retention time'])])
# Setup the dataset for dlomix
train_data = RetentionTimeDataset(data_source=train_array,
                                  seq_length=sequence_length,
                                  batch_size=batch_size).train_data
test_data = RetentionTimeDataset(data_source=test_array,
                                 seq_length=sequence_length,
                                 batch_size=batch_size).train_data

In [9]:
from dlomix.models.prosit import PrositRetentionTimePredictor
# Import the Prosit model, compile and train it
model = PrositRetentionTimePredictor(seq_length=sequence_length)
model.compile(loss='mse', metrics=['mae'], optimizer='adam')
epochs=10
history = model.fit(x=train_data, validation_data=test_data, epochs=epochs)

Epoch 1/10
 28/305 [=>............................] - ETA: 4:20 - loss: 0.0670 - mae: 0.2062

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(epochs), history.history['loss'], '-', color='r', label='Training loss')
plt.plot(range(epochs), history.history['val_loss'], '--', color='r', label='Validation loss')
plt.plot(range(epochs), history.history['mae'], '-', color='b', label='Training accuracy')
plt.plot(range(epochs), history.history['val_mae'], '--', color='b', label='Validation accuracy')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()